pip install pymssql 



In [1]:
import pandas as pd
import pymssql 

In [2]:
server="SQL-retail2.nikamed.local"
base="Retail2_shops"

In [101]:
sql="""
with mag_iskl as (
SELECT CAST(_IDRRef AS uniqueidentifier) AS Ссылка, --исключение техн склада
_IDRRef, 
_Code AS Код
FROM _Reference10
where _Code='0-002'
),
--------------------------------------исх продажи-----------------------
sales_ish as (
select 
       cast(dateadd(year, -2000, d21._date_time) as date) as Дата , 
         
       
       iif(e._EnumOrder=1,'ВозвратДВД','Продажи') as 'ВидОперации'
       ,iif(e._EnumOrder=1, -d21t._Fld6384, d21t._Fld6384 ) as 'Количество'
       ,iif(e._EnumOrder=1, -d21t._Fld6397, d21t._Fld6397) as 'Сумма'
        
              , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
        
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), 
                                  cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
        

from _Document21_VT6378 d21t

left outer join _Document21 d21 on d21t._Document21_IDRRef = d21._IDRRef /*исх чек*/

LEFT OUTER JOIN _reference47 r47 ON d21._Fld6331RRef = r47._idrref /*вид документа*/
LEFT OUTER JOIN _Enum352 e ON d21._Fld6333RRef = e._idrref /*вид документа в перечислениях чека ккм - 0-продажа, 1-возврат*/

left outer join _Reference10 r10 on d21._Fld6339RRef = r10._idrref /*shop*/


left outer join  _Document21 d2_21 on d21._Fld6353RRef=d2_21._idrref /*поиск мотив маг*/
left outer join _Reference10 r2_10 on d2_21._Fld6339RRef = r2_10._idrref /*shop*/

where d21._marked = 0 and d21._Posted = 1
and d21._date_time >= '08.01.4021' 


union all

select 
       cast(dateadd(year, -2000, d._date_time) as date) as 'Дата', 
       
        N'Возврат'
       , -dt._Fld3984 as 'Количество'
       , -dt._Fld3988 as 'СуммаЧека'
       
       , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
        
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
        

from _Document216_VT3980 dt

left outer join _Document216 d on dt._Document216_IDRRef = d._IDRRef /*исх возвратный чек*/

left outer join _Reference10 r10 on d._Fld3950RRef = r10._idrref /*shop*/
left outer join  _Reference172 r172  on dt._Fld4001RRef = r172._idrref
left outer join  _Reference23 r23  on dt._Fld3982RRef = r23._idrref


left outer join _Reference131 r131 on d._Fld11137RRef= r131._idrref --  склады
left outer join _Reference10 r2_10 on r131._Fld2686RRef = r2_10._idrref /*shop к складу*/

where d._marked = 0 and d._Posted = 1
and d._date_time >= '08.01.4021'
),
-------------суммарные продажи------------------------------------
sales_sum  as(
SELECT Дата as date_ss, sum(Сумма) as Summa, ccМагазин, ccМагазинМотивационный FROM sales_ish
group by Дата,  ccМагазин,ccМагазинМотивационный
)


SELECT 
cast(dateadd(year, -2000, i._fld10076) as date) AS date_ii,
sum(_fld10077) AS Трафик,
cast( r._Fld2907RRef as uniqueidentifier) AS cсМагазин,
sum(sales_sum.Summa) as Summa_ii

FROM _InfoRg10074 i 


left outer JOIN _reference142 r ON i._Fld10075RRef = r._IDRRef
left outer join mag_iskl mi on r._Fld2907RRef=mi._IDRRef
left outer join sales_sum  on sales_sum.date_ss=(cast(dateadd(year, -2000, i._fld10076) as date)) 
        and sales_sum.ccМагазин=cast( r._Fld2907RRef as uniqueidentifier)

WHERE i._fld10076 > '08.01.4021'
AND _fld10077 > 0
and mi._IDRRef is null
group by cast(dateadd(year, -2000, i._fld10076) as date) , cast( r._Fld2907RRef as uniqueidentifier)
"""

In [102]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,charset =  'cp1251',database = base ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

In [103]:
%%time
df_tr=read_sql(sql,base, server)

Wall time: 3.87 s


In [104]:
sum(df_tr['Трафик'])

718743.0

In [105]:
df_tr

,date_ii,Трафик,cсМагазин,Summa_ii
0,2021-09-13,61.0,4a3ced86-7492-28df-11e6-dd53a25ffe10,1093400.0
1,2021-08-05,44.0,4a3c988b-7492-28df-11e5-6cc2b15a7e3a,1262447.0
2,2021-09-21,57.0,4a3c7b92-7492-28df-11e6-97839a976d1f,3126200.0
3,2021-08-11,47.0,1100b287-72d8-8824-11df-10ed9da89bc6,4828788.0
4,2021-08-31,9.0,4a3ced86-7492-28df-11e6-dd5b1cd79d07,169432.0
...,...,...,...,...
11424,2021-09-19,47.0,4a3cafb9-7492-28df-11e7-203689cf9bf9,2428294.0
11425,2021-09-12,50.0,1500b98d-015d-002a-11ea-0a1f62408fe9,1083744.0
11426,2021-08-26,41.0,4a3cf985-7492-28df-11e5-2c544f5c615a,1532280.0
11427,2021-08-02,65.0,4a3c58b9-7492-28df-11e6-240486996548,3860544.0


conn.close()

In [95]:
%%time
df_tr=read_sql(sql,base, server)


Wall time: 540 ms


In [96]:
sum(df_tr['Трафик'])

690385.0

In [97]:
df_tr


,date_ii,Трафик,cсМагазин
0,2021-09-16,70.0,4a3ca086-7492-28df-11e1-26e56017ec38
1,2021-09-03,37.0,1500578d-015d-0b43-11e9-c998961af1c0
2,2021-08-23,83.0,4a3ced86-7492-28df-11e6-dd56e64b7598
3,2021-09-03,107.0,4a3ccdba-7492-28df-11e4-5b50ef7829d1
4,2021-09-11,49.0,4a3cc085-7492-28df-11e6-137b34be108f
...,...,...,...
11424,2021-08-12,46.0,1500458f-015d-032a-11eb-0c760711c161
11425,2021-09-27,43.0,4a3c3cb8-7492-28df-11e7-778fe6fa7541
11426,2021-08-14,42.0,4a3cf985-7492-28df-11e5-2c544f5c615a
11427,2021-09-03,36.0,1500808c-015d-0443-11e9-29fddcf9d848


In [ ]:
sql="""


with sales_ish as(
select 
       cast(dateadd(year, -2000, d21._date_time) as date) as 'Дата' , 
         
       
       iif(e._EnumOrder=1,'ВозвратДВД','Продажи') as 'ВидОперации'
       ,iif(e._EnumOrder=1, -d21t._Fld6384, d21t._Fld6384 ) as 'Количество'
       ,iif(e._EnumOrder=1, -d21t._Fld6397, d21t._Fld6397) as 'Сумма'
        
              , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
        
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), 
                                  cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
        

from _Document21_VT6378 d21t

left outer join _Document21 d21 on d21t._Document21_IDRRef = d21._IDRRef /*исх чек*/

LEFT OUTER JOIN _reference47 r47 ON d21._Fld6331RRef = r47._idrref /*вид документа*/
LEFT OUTER JOIN _Enum352 e ON d21._Fld6333RRef = e._idrref /*вид документа в перечислениях чека ккм - 0-продажа, 1-возврат*/

left outer join _Reference10 r10 on d21._Fld6339RRef = r10._idrref /*shop*/


left outer join  _Document21 d2_21 on d21._Fld6353RRef=d2_21._idrref /*поиск мотив маг*/
left outer join _Reference10 r2_10 on d2_21._Fld6339RRef = r2_10._idrref /*shop*/

where d21._marked = 0 and d21._Posted = 1
and d21._date_time >= '08.01.4021' 


union all

select 
       cast(dateadd(year, -2000, d._date_time) as date) as 'Дата', 
       
        N'Возврат'
       , -dt._Fld3984 as 'Количество'
       , -dt._Fld3988 as 'СуммаЧека'
       
       , cast(r10._idrref as uniqueidentifier) as 'ccМагазин'
        
       , iif(r2_10._Code is null, cast(r10._idrref as uniqueidentifier), cast(r2_10._idrref as uniqueidentifier) ) as 'ccМагазинМотивационный'
        

from _Document216_VT3980 dt

left outer join _Document216 d on dt._Document216_IDRRef = d._IDRRef /*исх возвратный чек*/

left outer join _Reference10 r10 on d._Fld3950RRef = r10._idrref /*shop*/
left outer join  _Reference172 r172  on dt._Fld4001RRef = r172._idrref
left outer join  _Reference23 r23  on dt._Fld3982RRef = r23._idrref


left outer join _Reference131 r131 on d._Fld11137RRef= r131._idrref --  склады
left outer join _Reference10 r2_10 on r131._Fld2686RRef = r2_10._idrref /*shop к складу*/

where d._marked = 0 and d._Posted = 1
and d._date_time >= '08.01.4021'
),
-------------суммарные продажи------------------------------------
sales_sum  as(
SELECT Дата, sum(Сумма) as Summa, ccМагазин, ccМагазинМотивационный FROM sales_ish
group by Дата,  ccМагазин,ccМагазинМотивационный
)

select * from sales_sum
--select * from sales_ish
"""

In [ ]:
def read_sql(sql,base, serv):
    #with pymssql.connect(server=serv ,database = base ,charset =  'cp1251') as  conn:
    with pymssql.connect(server=serv ,database = base ) as  conn:                  
    
        cursor = conn.cursor()  
        df = pd.read_sql( sql,conn)
    return df

In [ ]:
%%time
df=read_sql(sql,base, server)

In [ ]:
sum(df['Summa'])


In [ ]:
df

In [ ]:
sum(df['Сумма'])

In [ ]:
df
